In [2]:
import pandas as pd
import numpy as np

In [3]:
source = 'Bitstamp'
crypto = 'XRP'
market = 'USD'
year = '2021'
dur_type = 'minute'
name = f'{source}_{crypto}{market}_{year}_{dur_type}.csv'
df = pd.read_csv(f'../data/{name}', skiprows=1, parse_dates=True)
df = df[['date', 'open', 'high', 'low', 'close', f'Volume {crypto}']]

In [4]:
df.head()

,date,open,high,low,close,Volume XRP
0,2021-11-13 07:13:00,1.18774,1.18774,1.18774,1.18774,0.000000
1,2021-11-13 07:12:00,1.18746,1.18774,1.18736,1.18774,10039.680000
2,2021-11-13 07:11:00,1.18754,1.18755,1.18754,1.18755,2937.000000
3,2021-11-13 07:10:00,1.18757,1.18800,1.18757,1.18800,4752.198398
4,2021-11-13 07:09:00,1.18744,1.18789,1.18744,1.18745,6207.214000


In [5]:
cols_to_norm = ['open', 'high', 'low', 'close', f'Volume {crypto}']
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [6]:
df.sort_values('date', inplace=True)

In [7]:
df.head()

,date,open,high,low,close,Volume XRP
455472,2021-01-01 00:01:00,0.003723,0.002940,0.006702,0.004234,0.003462
455471,2021-01-01 00:02:00,0.004180,0.003032,0.007159,0.004536,0.001358
455470,2021-01-01 00:03:00,0.004083,0.002923,0.007022,0.004382,0.005899
455469,2021-01-01 00:04:00,0.004175,0.003060,0.007153,0.004559,0.004538
455468,2021-01-01 00:05:00,0.004078,0.002912,0.006633,0.003994,0.001968


In [8]:
df = df.loc[(df['date'] > '2021-01-01') & (df['date'] <= '2021-01-05')]

In [9]:
df.iloc[100:200]

,date,open,high,low,close,Volume XRP
455372,2021-01-01 01:41:00,0.006048,0.005144,0.009008,0.006545,0.001882
455371,2021-01-01 01:42:00,0.006036,0.004996,0.009014,0.006499,0.000692
455370,2021-01-01 01:43:00,0.006162,0.004996,0.009139,0.006499,0.000000
455369,2021-01-01 01:44:00,0.006562,0.005601,0.009539,0.007104,0.002532
455368,2021-01-01 01:45:00,0.006916,0.005767,0.009893,0.007269,0.003953
...,...,...,...,...,...,...
455277,2021-01-01 03:16:00,0.006191,0.005024,0.009059,0.006476,0.002580
455276,2021-01-01 03:17:00,0.006350,0.005230,0.009259,0.006619,0.000502
455275,2021-01-01 03:18:00,0.006299,0.005133,0.009276,0.006636,0.000609
455274,2021-01-01 03:19:00,0.006350,0.005184,0.009145,0.006590,0.003239


In [10]:
df.iloc[0:100]

,date,open,high,low,close,Volume XRP
455472,2021-01-01 00:01:00,0.003723,0.002940,0.006702,0.004234,0.003462
455471,2021-01-01 00:02:00,0.004180,0.003032,0.007159,0.004536,0.001358
455470,2021-01-01 00:03:00,0.004083,0.002923,0.007022,0.004382,0.005899
455469,2021-01-01 00:04:00,0.004175,0.003060,0.007153,0.004559,0.004538
455468,2021-01-01 00:05:00,0.004078,0.002912,0.006633,0.003994,0.001968
...,...,...,...,...,...,...
455377,2021-01-01 01:36:00,0.006288,0.005333,0.009265,0.006836,0.003206
455376,2021-01-01 01:37:00,0.006259,0.005093,0.009236,0.006596,0.001848
455375,2021-01-01 01:38:00,0.006345,0.005236,0.009322,0.006710,0.003379
455374,2021-01-01 01:39:00,0.006373,0.005207,0.009351,0.006710,0.000000


In [11]:
def rsi(df, periods = 14, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [12]:
df['rsi'] = rsi(df)

In [13]:
df.head()

,date,open,high,low,close,Volume XRP,rsi
455472,2021-01-01 00:01:00,0.003723,0.002940,0.006702,0.004234,0.003462,NaN
455471,2021-01-01 00:02:00,0.004180,0.003032,0.007159,0.004536,0.001358,NaN
455470,2021-01-01 00:03:00,0.004083,0.002923,0.007022,0.004382,0.005899,NaN
455469,2021-01-01 00:04:00,0.004175,0.003060,0.007153,0.004559,0.004538,NaN
455468,2021-01-01 00:05:00,0.004078,0.002912,0.006633,0.003994,0.001968,NaN
